In [13]:
import numpy as np
import scipy as sp

In [14]:
from quadrotor.dynamics import QuadrotorDynamicsBase, QuadrotorState
from quadrotor.controller import ControllerBase, QuadrotorCommands
from quadrotor.trajectory import TrajectoryBase, TrajectoryState

In [15]:
from sym import Rot3

def dRot3(R: Rot3, omega: np.ndarray) -> Rot3:
    """
    NOTE(roy): If interested, we can discuss the math here later. There are other
    ways of doing this but to keep the ode_int formulation, we specifically require
    dq/dt. Refer to 'Baseile Graf - Quaternions and Dynamics' or (for intuition)
    https://math.stackexchange.com/questions/1896379/how-to-use-the-quaternion-derivative
    """
    quat = np.array(R.to_storage())  # Convert to raw quaternion
    (q0, q1, q2, q3) = quat  # xyzw storage, see https://symforce.org/api-gen-cpp/class/classsym_1_1Rot3.html

    G = np.array([[ q3,  q2, -q1, -q0],
                [-q2,  q3,  q0, -q1],
                [ q1, -q0,  q3, -q2]])
    quat_dot = (G.T @ omega) / 2
    return Rot3.from_storage(quat_dot)

In [16]:
from dataclasses import dataclass, field

@dataclass
class Params:
    """A little utility class to hold our quadrotor parameters"""
    mass: float = 1.352  # [kg]

    # NOTE(roy) this is a matrix now as discussed!
    # NOTE(roy) don't get scared by the field/default_factory thing! It's just a way to do default field initialization for dataclasses,
    # see https://docs.python.org/3/library/dataclasses.html. Don't worry about it!
    inertia: np.ndarray = field(default_factory=lambda: np.array([
        [9.8e-3, 0, 0],
        [0, 10.02e-3, 0],
        [0, 0, 18.6e-3],
    ]))

    rotor_diameter: float = 10 * 0.0254  # [m] 10 inches

    static_thrust_coefficient: float = 0.14553  # [-] Thrust coefficient
    static_torque_coefficient: float = 0.01047  # [-] Torque coefficient

    # NOTE(roy) assume this is the same front-to-back and left-to-right
    arm_length: float = 0.3814 / 2.0  # [m] Arm length

    # Universal constants
    g: float = 9.80665  # [m/s2] Gravity
    rho: float = 1.225  # [kg/m3] Sea-level density

    def rotor_model(self, static_coefficient: float) -> float:
        # NOTE(roy) I'm just including this for completeness sake, this means we can e.g. see how air density
        # influences our simulation
        # Refer to this source for derivation: https://web.mit.edu/16.unified/www/FALL/thermodynamics/notes/node86.html
        return self.rho * static_coefficient * (self.rotor_diameter ** 4) / (4 * np.pi ** 2)

    # NOTE(roy) if you're unsure what @property decorators do: they make it so that we can call k_thrust as if it was just a class
    # variable. I.e. we'd write params.k_thrust now, rather than params.k_thrust()
    @property
    def k_thrust(self):
        return self.rotor_model(self.static_thrust_coefficient)

    @property
    def k_torque(self):
        return self.rotor_model(self.static_torque_coefficient)

# Simply treat as a global
p = Params()

### Finishing the controller
In this notebook, you'll take your previous implementation where we just controller altitude (z-position) and expand it to build the full position-attitude loop.

**Note that I updated the controller params class with some extra parameters!**

#### Recommended steps
- Copy your 3D quadrotor model
- Copy your controller implementation
- Expand the controller class
    - Compute the desired roll/pitch using the equations from the lecture
    - Compute the attitude errors
    - Compute the desired angular acceleration
    - Compute `u_2`
    - Tune your system (the gains provided _should_ provide a relatively stable starting point!)

### The dynamics

In [17]:
def whatami(x) -> str:
        """This function prints some basic info about a numpy ndarray for troubleshooting purposes"""
        return 'is a ' + str(x.__class__.__name__) + ' with shape ' + str(x.shape) + ' = ' + str(x)

class FullQuadrotorDynamics(QuadrotorDynamicsBase):
    def __init__(self) -> None:
        # NOTE(roy): I advise you to scroll through dynamics.py and see what the __init__
        # function in the base class does!
        super().__init__()


    @staticmethod
    def rotor_thrust_model(rotor_rates: np.ndarray) -> np.ndarray:
        """Accepts an array of rotor rates and returns an array of thrusts [F1, F2, F3, F4]"""
        # NOTE(roy): Implement the simple rotor trust model we discussed in the slides
        # To see how complex this can get, see for example: https://arxiv.org/pdf/1601.00733.pdf
        # force of each rotor is related to rotor rates squared times constant p.k_thrust
        thrusts =  rotor_rates * rotor_rates * p.k_thrust     
        return thrusts


    def step(self, t: float, input: QuadrotorCommands) -> QuadrotorState:
        F1, F2, F3, F4 = FullQuadrotorDynamics.rotor_thrust_model(input.rotor_rates)
        L = p.arm_length
        m_t = p.k_torque / p.k_thrust     # multiply the rotor_forces by this to get the torques

        mixing_matrix = np.array([
            [  1,      1,      1,        1],
            [  0,      L,      0,      - L],
            [- L,      0,      L,        0],
            [m_t,  - m_t,    m_t,    - m_t]
        ])

        u = mixing_matrix @ np.array([F1, F2, F3, F4])         # thrust and torques = mixing_matrix @ rotor_thrusts
        # if t == 0: print("u", whatami(u))                      # u is a ndarray with shape (4,) - a horizontal vector [thrust, MomentX, MomentY, MomentZ]

        u1 = u[0]                                               # thrust is first element
        # if t == 0: print("u1", whatami(u1))                    # u1 is a numpy.float64 with shape () = 10.237126413876851

        u2 = u[1:4]                                             # torques (aka moments) are the last three elements
        # if t == 0: print("u2", whatami(u2))                    # u2 is a ndarray with shape (3,) = [ 1.38783753e-19 -1.38783753e-19  0.00000000e+00]

         # NOTE(roy) this local function is a wrapper around the class state_derivative function
        # all this does is convert the our state dataclass from/into a vector that the scipy
        # integration solver can work with. See dynamics.py for the definition of these
        # from/to state vector functions.
        def state_derivative_wrapped(t: float, state: np.ndarray) -> np.ndarray:
            return FullQuadrotorDynamics.state_derivative(
                t,
                QuadrotorState.from_state_vector(state),
                u1,
                u2
            ).to_state_vector()
           
        state_vector = self.state.to_state_vector()
        # if t == 0: print('start state_vector =', state_vector)  # state_vector = [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
        # if t == 0: print('start self.state =', self.state)      # self.state = QuadrotorState(position=array([0., 0., 0.]), orientation=<Rot3 [0.0, 0.0, 0.0, 1.0]>, velocity=array([0, 0, 0]), angular_velocity=array([0., 0., 0.]))

        solution = sp.integrate.solve_ivp(state_derivative_wrapped, (0, self.dt), state_vector)
        self.state = QuadrotorState.from_state_vector(solution['y'][:, -1])
        # if t > 3.99: print('at time', t, 'state_vector =', state_vector)  # state_vector = [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
        # if t > 3.99: print('at time', t, 'self.state =', self.state)      # self.state = QuadrotorState(position=array([0., 0., 0.]), orientation=<Rot3 [0.0, 0.0, 0.0, 1.0]>, velocity=array([0, 0, 0]), angular_velocity=array([0., 0., 0.]))
        return self.state
    
    
    @staticmethod
    def state_derivative(t: float, state: QuadrotorState, u1: np.ndarray, u2:np.ndarray) -> QuadrotorState:
        R = state.orientation           # a Rot3 (SymForce quaternion) such as <Rot3 [1.0, 3.0, 2.0, 1.0]>
        w = state.angular_velocity      # (3,) horizontal vector
        I = p.inertia                   # (3, 3) matrix
        actual_weight_ratio = 1         # change this to more than 1 to simulate if drone is heavier than expected due to stickers, attachments, mud, etc

        # acceleration = downward gravity + (thrust rotated from body-frame to world-frame) /  mass
        accel = (np.array([0, 0, -  p.g]) + R * np.array([0, 0, u1])) / (p.mass * actual_weight_ratio)       # (3,) = (3,) + Rot3 * (3,)
        
        # this is from project session 3, page 38:
        # omega_dot = I_inverse @ (M - omega CrossProduct Iomega)
        # (3,) = (3,3) @ ((3,) - crossproduct((3,), (3,3) @ (3,)))
        # angular_accel = np.linalg.inv(I) @ (M - np.cross(w, I @ w))     # (3,)   a vector [rotationx, rotationy, rotationz]
        angular_accel = np.linalg.inv(I) @ (u2 - np.cross(w, I @ w))     # (3,)   a vector [rotationx, rotationy, rotationz] Roy equation, works



        # NOTE(roy) this might look a little confusing at first glance, but we're populating DERIVATIVES
        # of the state here! So position -> velocity, velocity -> acceleration, etc...
        # If you can think of a more readable way to do this (e.g. defining a QuadrotorStateDerivative class)
        # feel free to open a pull-request into the upstream repository!
        return QuadrotorState(
            position=state.velocity,
            velocity=accel,
            orientation=dRot3(state.orientation, state.angular_velocity),
            angular_velocity= angular_accel
        )


### The controller

In [18]:
@dataclass
class ControllerParams:
    """The same as above but for the controller gains"""

    # NOTE(roy) diag just means we get a matrix with these values on the diagonal,
    # so these are 3x3 matrices (print them if you're unsure!)
    # We use this formulation because it makes the math a bit more readable later on!
    # K_p is proportional gain (the spring)
    K_p: np.ndarray = field(default_factory=lambda: np.diag([1, 1, 100]))       # the last number is what Roy described as the K_p
                                                                                # [[  1   0   0]
                                                                                #  [  0   1   0]
                                                                                #  [  0   0 100]]   
    
    # K_d is derivative gain (the damper)
    K_d: np.ndarray = field(default_factory=lambda: np.diag([0.5, 0.5, 30]))    # the last number is what Roy described as the K_d
                                                                                # [[ 0.5  0.   0. ]
                                                                                #  [ 0.   0.5  0. ]
                                                                                #  [ 0.   0.  30. ]]

    # Added gains for the attitude controller
    Ka_p: np.ndarray = field(default_factory=lambda: np.diag([300, 300, 50]))
                                                                                # [[300   0   0]
                                                                                #  [  0 300   0]
                                                                                #  [  0   0  50]]    
    Ka_d: np.ndarray = field(default_factory=lambda: np.diag([50, 50, 20]))
                                                                                # [[50  0  0]
                                                                                #  [ 0 50  0]
                                                                                #  [ 0  0 20]]
    # Min/max rotor rates!
    rotor_rate_min: float = 180
    rotor_rate_max: float = 600

controller_p = ControllerParams()

print('K_p =', controller_p.K_p[2,2])
print('K_d =', controller_p.K_d[2,2])
print('ratio =', controller_p.K_d[2,2] / (2 * np.sqrt(p.mass * controller_p.K_p[2,2])))
# this is the Damping ration on Project Session 6 page 8.
# it's a little unclear if 1 or sqrt(2) is "critically damped", but 1.29 is "slightly overdamped", 0.2 is "massively underdamped"
# overdamped will be slower to respond, but nice and smooth. underdamped will be the crazy springy overshooting and undershooting
print('Ka_p =', controller_p.Ka_p[2,2])
print('Ka_d =', controller_p.Ka_d[2,2])
print('ratio =', controller_p.Ka_d[2,2] / (2 * np.sqrt(p.mass * controller_p.Ka_p[2,2])))


K_p = 100
K_d = 30.0
ratio = 1.2900392177883402
Ka_p = 50
Ka_d = 20
ratio = 1.2162606385262997


In [19]:
class Controller(ControllerBase):
    @staticmethod
    def rotor_rates_from_u(u: np.ndarray) -> np.ndarray:
        """This function accepts an array u [desired_thrust, desired_torqueX, desired_torqueY, desiqured_torqueZ]
           and returns an array of commanded_rotor_rates [w1, w2, w3, w4]"""
        # NOTE(roy) implement your conversion from u (so that's [u1, u2_1, u2_2, u2_3] like we discussed, a 4-sized vector)
        # to rotor rates (which is also a four-sized vector)

        L = p.arm_length
        m_t = p.k_torque / p.k_thrust     # multiply the rotor_forces by this ratio to get the rotor_torques
        mixing_matrix = np.array([
            [  1,      1,      1,        1],
            [  0,      L,      0,      - L],
            [- L,      0,      L,        0],
            [m_t,  - m_t,    m_t,    - m_t]
        ])

        # NOTE(roy) the steps here are:
        #  1. convert u(desired thrust and torques) -> required_rotor_forces with the inverse mixing matrix

        # we are going to do the inverse of the dynamics equation, which was this:
        # u(desired_thrust_and_torques) (4,) = mixing_matrix (4,4) @ required_rotor_forces (4,)
        # [desired_thrust, desired_torqueX, desired_torqueY, desired_torqueZ] = mixing_matrix @ [F1, F2, F3, F4]

        # divide both sides by the mixing matrix so new equation will be:
        # required_rotor_forces (4,) = inverse_mixing_matrix (4,4) @ u(desired_thrust_and_torques) (4,)
        # [F1, F2, F3, F4] = inverse_mixing_matrix (4,4) @ [desired_thrust, desired_torqueX, desired_torqueY, desired_torqueZ]

        required_rotor_forces = np.linalg.inv(mixing_matrix) @ u       # (4,) = (4,4) @ (4,) 

        # if any required_rotor_forces are NEGATIVE, set them to zero so it does not crash the square root function
        commanded_rotor_forces = np.maximum(required_rotor_forces, 0)

        #  2. convert required_rotor_forces -> commanded_rotor_rates by inverting the rotor rate model
        # F = p.k_thrust * w^2      required_rotor_forces =  rotor_rates * rotor_rates * p.k_thrust  (in rotor_thrust_model)
        # w^2 = required_rotor_forces / p.k_thrust        
        # w = square_root(required_rotor_forces / p.k_thrust)
        commanded_rotor_rates = np.sqrt(commanded_rotor_forces / p.k_thrust)        # (4,) = np.sqrt((4,) / float)

        #  3. limit the rotor rates with rotor_rate_min/rotor_rate_max above!
        commanded_rotor_rates = np.clip(commanded_rotor_rates, controller_p.rotor_rate_min, controller_p.rotor_rate_max)

        # print('rotor_rates =', whatami(commanded_rotor_rates))
        return commanded_rotor_rates


    def step(self, t: float, trajectory: TrajectoryState, state: QuadrotorState) -> QuadrotorCommands:
        printing = 0
        # printing = t == 0 or t > 10.0 - .01
        desired_position = trajectory.position
        desired_velocity = trajectory.velocity
        actual_position = state.position
        actual_velocity = state.velocity
        if printing:
            print('---------------------------------------')
            print('time =', t)
            print('desired_position =', desired_position)
            print('desired_velocity =', desired_velocity)
            print('actual_position =', actual_position)
            print('actual_velocity =', actual_velocity)
        
        # in the dynamics model, the flow of information is rotor_rates -> rotor_thrusts -> body_frame_thrust_and_torques -> world_frame_thrust_and torques
        # we need to go the other way: desired_world_frame_thrust_and_torques -> desired_body_frame_thrust_and_torques -> desired_rotor_thrusts -> required_rotor_rates
        
        # NOTE(roy) first compute the desired acceleration using the equation from the lectures!
        # I already extracted the values you need to compute it above
        # project session 5 page 17 equation:
        # r¨= - K_d @ (trajectory.velocity - state.velocity) - K_p @ (trajectory.position - state.position)
        # desired_acceleration (3,) = - damper_gain (3,3) @ velocity_error (3,) - spring_gain (3,3) @ position_error  (3,)

        desired_acceleration = controller_p.K_d @ (desired_velocity - actual_velocity) + controller_p.K_p @ (desired_position - actual_position)
        # somehow the equations did not work unless I got rid of the minus signs before each term
        if printing: print('desired_acceleration =', desired_acceleration)    # 

        # NOTE(roy) now compute u_1 from the desired_acceleration using the linearized equation from the lectures!
        # project session 5 page 18 says, assuming R is at identity (drone pointing straigt up),
        # u_1 = m * (g + r¨z_des), thrust = mass times (gravity + desired z acceleration)
        u_1 = p.mass * (p.g + desired_acceleration[2])    # required thrust is the z value of desired_acceleration: should be a float
        if printing: print('u_1 =', u_1)

        # IMPLEMENT THE TORQUES! see page 13 of project session 6 and 14:00 into the recording of week 7 meeting!
        # using the "small angle assumption" that sin(x) ~ x for small x  (this will not work with big angles though)
        # roll and pitch travel with yaw, they rotate along with it

        yaw, pitch, roll = state.orientation.to_yaw_pitch_roll()
        find_desired_roll_and_pitch = np.array([[np.cos(yaw),    np.sin(yaw)],
                                                [np.sin(yaw),   -np.cos(yaw)]])
        if printing: print('desired_acceleration[0:2] =', desired_acceleration[0:2])

        desired_pitch, desired_roll  = find_desired_roll_and_pitch.T @ desired_acceleration[0:2]
        ############################# AFTER SWAPPING THESE SO PITCH IS FIRST AND ROLL IS SECOND, IT WORKS!


        if printing:
            print('desired_roll =', desired_roll)
            print('desired_pitch =', desired_pitch)
            print('desired yaw =', trajectory.yaw)


        yaw_error = trajectory.yaw - yaw
        yaw_error = (yaw_error + np.pi) % (2 * np.pi) - np.pi       # keep it in the domain of [-pi, pi]

        roll_error = desired_roll - roll
        roll_error = (roll_error + np.pi) % (2 * np.pi) - np.pi       # keep it in the domain of [-pi, pi]

        pitch_error = desired_pitch - pitch
        pitch_error = (pitch_error + np.pi) % (2 * np.pi) - np.pi       # keep it in the domain of [-pi, pi]


        ##################### TESTING SOMETHING NEW ROY SUGGESTED - try to just have yaw pitch roll all go to zero
        # yaw_error = 0 - yaw
        # pitch_error = 0 - pitch
        # roll_error = 0 - roll

        if printing: print(f'roll={roll}      pitch={pitch}      yaw={yaw}')
        
        attitude_error = np.array([roll_error, pitch_error, yaw_error])
        if printing: print('attitude_error =', attitude_error)

        angular_velocity_error = np.array([0, 0, trajectory.yaw_rate]) - state.angular_velocity
        if printing:
            print('state.angular_velocity =', state.angular_velocity)
            print('desired angular velocity =', (0, 0, trajectory.yaw_rate))
            print('angular_velocity_error =', angular_velocity_error)
        
        
        # do exact same thing as previous lecture using the gains (this time use angular gains)
        # for angular rate for roll and pitch, keep it relatively simple, set to zero, damp the system, keep as level as possible. penalize any amount of angular velocity

        # use the equations on page 18 and 29 of project session 6 to get the commanded angular acceleration,
        # which we can then easily convert into 3 body torques, because we know the angular acceleration @ the inertia matrix = the torques   
        # Roy said (M = I @ alpha) ????? alpha is omega dot? I found this on a website talking about torques: α is angular acceleration (rate of change of angular velocity)
        # u_2 is the intertia matrix times the error term
        # use Ka_d (attitude derivative gain) and Ka_p (attitude proportional gain)
        
        u_2 = p.inertia @ (controller_p.Ka_d @ angular_velocity_error + controller_p.Ka_p @ attitude_error)
        if printing: print('u_2 =', u_2)

        u = np.array([u_1, *u_2])       # (4,)      [desired_thrust, 0, 0, 0]
        if printing: print('u=', u)

        commanded_rotor_rates = Controller.rotor_rates_from_u(u)
        return QuadrotorCommands(commanded_rotor_rates)

### The trajectory
Just something simple, mess around with it!

In [20]:
class JumpTrajectory(TrajectoryBase):
    def eval(self, t: float):
        x = 0 if t < 1.0 or t > 5.0 else 1
        y = 0 if t < 2.0 or t > 5.0 else 1
        z = 0
        # z = np.sin(t * 4) / 4
        # x, y, z = 0, 0, 0

        position = np.array([x, y, z])

        yaw_rate = 0.5 * 1
        yaw = yaw_rate * t * 0
        return TrajectoryState(t, position=position, yaw=yaw, yaw_rate=yaw_rate)


class CircleTrajectory(TrajectoryBase):
    def eval(self, t: float):
        x, y, z = 0, 0, 0
        yaw, yaw_rate = 0, 0
        max_radius = 1
        radius = 0
        blend_time = 2
        loop_time = 10
        blend = min(1, t / blend_time)
        radius = max_radius * blend
        yaw_rate = 2 * np.pi / loop_time
        yaw = yaw_rate * t + 3/4 * np.pi * blend
        x = radius * np.cos(2*np.pi / loop_time * t)
        y = radius * np.sin(2*np.pi / loop_time * t)
        z = 1 * blend

        position = np.array([x, y, z])
        return TrajectoryState(t, position=position, yaw=yaw, yaw_rate=0)       # yaw_rate = 0 fixes the strange angular offset I was getting!
        # return TrajectoryState(t, position=position, yaw=yaw, yaw_rate=yaw_rate)


class RoseCurveTrajectory(TrajectoryBase):
    def eval(self, t: float):
        x, y, z = 0, 0, 0
        yaw, yaw_rate = 0, 0
        loop_time = 10
        blend_time = 2
        blend = min(1, t / blend_time)
        theta = (2*np.pi / loop_time) * t
        radius = 1 * np.cos(3 * theta) * blend
        x = radius * np.cos(theta)
        y = radius * np.sin(theta)
        yaw = (4 * theta + 1/4 * np.pi) * blend
        # yaw = (np.arctan2(y, x) + 3/4 * np.pi)   #y,x   the camera tries to point at the origin, this is too jarring when it pops
        position = np.array([x, y, z])
        return TrajectoryState(t, position=position, yaw=yaw, yaw_rate=0)       # yaw_rate = 0 fixes the strange angular offset I was getting!
        

class RoseCurveVerticalTrajectory(TrajectoryBase):
    def eval(self, t: float):
        x, y, z = 0, 0, 0
        yaw, yaw_rate = 0, 0
        loop_time = 12
        blend_time = 2
        blend = min(1, t / blend_time)
        theta = (2*np.pi / loop_time) * t
        radius = .3 * np.cos(3 * theta) * blend
        x = radius * np.cos(theta + np.pi / 2) * blend
        z = radius * np.sin(theta + np.pi / 2) * 1 * blend
        yaw = -3/4 * np.pi * blend
        position = np.array([x, y, z])
        return TrajectoryState(t, position=position, yaw=yaw, yaw_rate=0)       # yaw_rate = 0 fixes the strange angular offset I was getting!


class DrunkTrajectory(TrajectoryBase):
    def eval(self, t: float):
        x, y, z = 0, 0, 0
        yaw, yaw_rate = 0, 0
        loop_time = 2
        blend_time = 2
        blend = min(1, t / blend_time)
        theta = (2*np.pi / loop_time) * t
        radius = .25
        x = radius * np.sin(theta) * blend
        y = (t + np.cos(4*t)) / 4
        z = radius * np.cos(2 * theta + np.pi * 2/3) * blend
        
        yaw = -3/4 * np.pi * blend
        position = np.array([x, y, z])
        return TrajectoryState(t, position=position, yaw=yaw, yaw_rate=0)       # yaw_rate = 0 fixes the strange angular offset I was getting!


In [21]:
from quadrotor.simulator import SimulatorBase, SimulatorState

# This might be slow to run the first time!
from quadrotor.renderer import animate_k3d

In [32]:
def run_and_render(initial_state: QuadrotorState, t_total: int = 2.0):
    sim = SimulatorBase(
        dt=0.005,
        dynamics=FullQuadrotorDynamics(),
        controller=Controller(),
        # trajectory=JumpTrajectory(),
        # trajectory=CircleTrajectory(),
        trajectory=RoseCurveTrajectory(),
        # trajectory=RoseCurveVerticalTrajectory(),
        # trajectory=DrunkTrajectory(),
        initial_state=initial_state,
        t_total=t_total,
    )
    
    # Run the simulator
    output = sim.simulate()

    # K3D renderer
    plot = animate_k3d(output)
    # plot = animate_k3d(output, t_lookahead=5.0)  # can set the lookahead to be longer than default 2
    # plot = animate_k3d(output, t_lookahead=1)  # can set the lookahead to be shorter than default 2
    
    plot.display()
    plot.start_auto_play()

In [33]:
# NOTE(roy) play with this variable and see what happens!
initial_state = QuadrotorState(
    position=np.array([0, 0, 0]),
    velocity=np.array([0, 0, 0]),
    orientation=Rot3.from_yaw_pitch_roll(0, 0, 0),
    angular_velocity=np.zeros(3),
)

run_and_render(initial_state, 10.0)

Output()

### Looking at the controller behavior
If all is well, the quadrotor should follow the trajectory!

Try some experiments:
- Can you make the controller more/less damped and faster/slower to respond?
- Can you write a trajectory that has feasible velocities/positions?
    - Hint: the trajectory that's currently there is infeasible, why?

### Commit your updated notebook to your fork!